# VigitelBot
Um robô construído em Python para mineração de dados do estudo Vigitel, do Ministério da Saúde.

Mais informações sobre o Vigitel em: 
http://portalms.saude.gov.br/o-ministro/938-saude-de-a-a-z/doencas-cronicas-nao-transmissiveis/17908-o-que-e-o-vigitel

Atenção - Mineração de Dados deve ser feita com ética e responsabilidade. 

Não modifique o código a fim de provocar uma sobrecarga nos servidores do Governo.

---

Marcelo Benedet Tournier - Chief Data Scientist - INOVAlife

Contato: marcelo@inova.life

LinkedIn: http://bit.ly/marcelotournier

In [93]:
# Instale selenium e importe os módulos necessários. 
# Você precisará instalar o chromedriver para uso headless. 
# Mais info em https://duo.com/decipher/driving-headless-chrome-with-python:

import selenium
from selenium import webdriver
from datetime import datetime
from pytz import timezone 
import urllib.request

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": "/path/to/download/dir",
  "download.prompt_for_download": False,
})

chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': "/Users/marcelotournier/Downloads"}}
command_result = driver.execute("send_command", params)

In [54]:
# Como eu moro em outro fuso horário, precisei ajustar o horario local para o fuso de Brasília com esta função:
  
def data_local():
    brasilia = timezone('America/Sao_Paulo')
    bsb_time = datetime.now(brasilia)
    return bsb_time.strftime('%Y%m%d')

In [55]:
# Lista de Capitais do Brasil, na ordem do site do Vigitel:
capitais = ['Aracaju',
 'Belem',
 'Belo Horizonte',
 'Boa Vista',
 'Campo Grande',
 'Cuiaba',
 'Curitiba',
 'Florianopolis',
 'Fortaleza',
 'Goiania',
 'Joao Pessoa',
 'Macapa',
 'Maceio',
 'Manaus',
 'Natal',
 'Palmas',
 'Porto Alegre',
 'Porto Velho',
 'Recife',
 'Rio Branco',
 'Rio de Janeiro',
 'Salvador',
 'Sao Luis',
 'Sao Paulo',
 'Teresina',
 'Vitoria',
 'Distrito Federal']

anos = range(2006,2017)

In [56]:
# Função para codificar a variável capital:
def code_cap(x):
    if x < 9:
        return 'c0'+str(x+1)
    else:
        return 'c'+str(x+1)

In [ ]:
# Minerando...
for numcap,cap in enumerate(capitais):
    for ano in range(2006,2018):

        # abrir o website
        driver.get("http://svs.aids.gov.br/bases_vigitel_viva/vigitel.php") #requesting the base url

        # Encontrar campos por id:
        id_nome = driver.find_element_by_id('nome')
        id_cpf = driver.find_element_by_id('cpf')
        id_end = driver.find_element_by_id('dir')
        id_cep = driver.find_element_by_id('cep')
        id_motivo = driver.find_element_by_id('motivo')
        id_loc = driver.find_element_by_id('loc') # iterar em cada elemento da lista "capitais"
        id_ano = driver.find_element_by_id('ano') # iterar entre (e incluindo) os anos 2006 a 2017

        # Encontrar o botão de envio:
        submit_button = driver.find_element_by_id('csv')

        # Preencher automaticamente os dados:
        id_nome.send_keys('Marcelo')
        id_cpf.send_keys('00000000001')
        id_end.send_keys('Av Geral SN São Paulo')
        id_cep.send_keys('00000000')
        id_motivo.send_keys('Pesquisa')
        id_loc.send_keys(cap.upper())
        id_loc.click()
        id_ano.send_keys(ano)
        id_ano.click()

        # Clicar na consulta:
        submit_button.click()

        # Baixar o arquivo:
        # Exemplo de url: http://svs.aids.gov.br/bases_vigitel_viva/tmp/temp_20181217_VIGITEL_MicroDado_2006_c05.CSV

        filename = 'temp_'+data_local()+'_VIGITEL_MicroDado_'+str(ano)+'_'+code_cap(numcap)+'.CSV'
        print('Parâmetros | Capital: ',cap.upper(),'Código: ',code_cap(numcap),'Ano: ',ano,)
        print('Baixar o arquivo:',filename)
        
        urllib.request.urlretrieve ("http://svs.aids.gov.br/bases_vigitel_viva/tmp/"+filename,filename) # metodo para Chrome headless

        print('Baixado com sucesso')